# Multithreading and Multiprocessing

### Threading
* typically, concurrency is created so that we can do some task while I/O is happening (e.g., a server can start processing a new request while waiting for data from a previous request to arrive)
* we can create objects that appear to be running independently, but simultaneously
* the job of threading is to enable an application to be responsive
* CPython, the default implementation of Python, has a Global Interpreter Lock (GIL), which prevents your application from doing two things at once, but rather, the CPU time is being rationed across your thread

### Simple threading example

In [1]:
from threading import Thread

class InputReader(Thread):
    """Thread example, extends Thread class"""

    def run(self):
        """
        Whatever is in the run method (or called from
        it) is executed in a separate thread
        """
        self.line_of_text = input('Enter some text: ')

input('Are you ready? When you hit return the thread will start.')
thread = InputReader() # create thread object
thread.start() # cf. thread.run() for no concurrency

count, result = 1, 1

while thread.is_alive():
    result = count * count
    count += 1

print('calculated squares up to {0} * {0} = {1:,}'.format(count, result))
print('while you typed "{}"'.format(thread.line_of_text))

Are you ready? When you hit return the thread will start.
Enter some text: Hello
calculated squares up to 2567130 * 2567130 = 6,590,151,302,641
while you typed "Hello"


In [1]:
# Does not work inside Jupyter
from threading import Thread
import json
from urllib.request import urlopen
import time

cities = ['Boulder', 'Atlanta', 'San Francisco',
          'Reno', 'Honolulu', 'Zurich', 'Dubai',
          'Dublin','Stuttgart', 'Rome']

class TempGetter(Thread):
    def __init__(self, city):
        """Initialize our thread

In the previous example, our class which extended Thread did not
need an __init__ method, because there was no per-thread information
to store. Which means that the __init__ method from the superclass
(Thread) was called automatically. Here, because we need to store
per-thread information (the city), we have to explicitly call the
__init__ method of Thread.
        """
        super().__init__()
        self.city = city

    def run(self):
        url_template = (
            'http://api.openweathermap.org/data/2.5/' 
            'weather?q={}&units=imperial'
                        '&&APPID=10d4440bbaa8581bb8da9bd1fbea5617')
        response = urlopen(url_template.format(self.city))
        data = json.loads(response.read().decode())
        self.temperature = data['main']['temp']
        
threads = [TempGetter(c) for c in cities] # creates 10 threads
start = time.time()

# start all 10 threads
for thread in threads:
    thread.start() # not run()

# wait for all 10 threads to complete
for thread in threads:
    thread.join()

for thread in threads:
    print("it is {0.temperature:.0f}°F in {0.city}"
          .format(thread))
print("Got {} temps in {} seconds"
      .format(len(threads), time.time() - start))

it is 48°F in Boulder
it is 73°F in Atlanta
it is 77°F in San Francisco
it is 56°F in Reno
it is 87°F in Honolulu
it is 51°F in Zurich
it is 78°F in Dubai
it is 78°F in Dublin
it is 46°F in Stuttgart
it is 71°F in Rome
Got 10 temps in 0.3241448402404785 seconds


In [2]:
%%bash
python3 getweather.py

python3: can't open file 'getweather.py': [Errno 2] No such file or directory


## Threading (cont'd)
* the main problem with threads is also their primary advantage–shared memory
 * all threads have access to all the memory
 * what if two threads access the same data?
* synchronization is the solution, but it's tricky
 * bugs due to incorrect synchronization can be very difficult to find due to ordering issues
* one solution is to force communication between threads to occur using a data structure that has built in locking, such as queue.Queue
* disadvantages could be outweighed by the fact that shared memory is FAST, except for the GIL

## Lab: threads
* create a program which uses threads to simulate a database server
* your "database server" should simply be a thread which sleeps for a random interval (check out `time.sleep()` and `random.randint()` if you're not familiar with them)
* your main thread should get input from the user and respond to it (perhaps reversing the input given by the user) while the database thread is busy

## Lab Solution: threads

Processing database request for 12 seconds
Finished processing database request
Processing database request for 13 seconds
Enter some text: hello
olleh deretne uoY 
Finished processing database request
Processing database request for 10 seconds
Finished processing database request
Processing database request for 14 seconds
Enter some text: ok
ko deretne uoY 
Finished processing database request
Processing database request for 15 seconds
Enter some text: qqq
Finished processing database request
mp has requested the database server stop running.


# Multiprocessing
* the Python multiprocessing library is designed for cases where CPU-bound jobs needs to happen in parallel and multiple cores are available
* advantages
 * separate memory space for each process
 * code is usually straightforward compared to threads
 * avoids GIL limitation
 * eliminates synchronization (assuming no shared memory)

## A Simple Multiprocessing Example

In [3]:
from multiprocessing import Process, cpu_count
import time
import os

class MuchCPU(Process):
    def run(self):
        print(os.getpid())
        print(__name__)
        for i in range(20_000_000):
            result = i * i

if __name__ == '__main__':
    print('Running...')
    procs = [MuchCPU() for f in range(cpu_count())]
    t = time.time()
    for p in procs:
        p.start()
    
    for p in procs:
        p.join()
    
    print('work took {} seconds'.format(time.time() - t))

Running...
64016
__main__
64017
__main__
64018
__main__
64019
__main__
64020
__main__
64021
64023
__main__
64022
__main__
__main__
work took 2.6905572414398193 seconds


## Multiprocessing (cont'd)
* no reason for more processes than there are processors
 * only `cpu_count()` procs can run simultaneously
 * each proc consumes resources with a full copy of Python interpreter
 * interproc communication is expensive
 * creating procs takes a nonzero amount of time
* so we create at most `cpu_count()` processes when the program starts and have them execute tasks as needed
* easy to implement a basic series of communicating processes to do this, but it can be tricky to debug, test, and get correct–we don't have to do all this work because the Python developers have already done it for us–multiprocessing pools


## Multiprocessing Pools
* pools abstract away the overhead of figuring out what code is running in main process and what code is running in subprocess
* abstraction restricts the number of places that code in different processes interact with each other, making it easier to keep track of
* pools also hide the passing of data between processes
 * using a pool looks much like a function call–you pass data into a function, it's executed in another process or processes, and when the work is complete, a value is returned
 * under the hood, a lot of work is being done to support this–objects in one process are being pickled (serialized) and passed into a pipe, then another process retrieves data from the pipe and unpickles it. Work is done in the subprocess and a result is produced. The result is pickled and passed into a pipe. Eventually, the original process unpickles it and returns it.

## Multiprocessing Pool Example

In [27]:
import random
import math
import os
from multiprocessing.pool import Pool

def prime_factor(value):
    factors = []
    print('prime_factor(', value, ')', os.getpid())
    for divisor in range(2, value-1):
        quotient, remainder = divmod(value, divisor)
        if not remainder:
            factors.extend(prime_factor(divisor))
            factors.extend(prime_factor(quotient))
            break
    else:
        factors = [value]
    return factors

if __name__ == '__main__':
    pool = Pool()

    to_factor = [
        random.randint(100_000, 50_000_000) for i in range(20)
    ]
    results = pool.map(prime_factor, to_factor)
    for value, factors in zip(to_factor, results):
        print("The factors of {} are {}".format(value, factors))

prime_factor( 14607445 ) 52771
prime_factor( 42534327 ) 52772
prime_factor( 8109461 ) 52770
prime_factor( 4985981 ) 52769
prime_factor( 7 ) 52769
prime_factor( 3 ) 52772
prime_factor( 5 ) 52771
prime_factor( 712283 ) 52769
prime_factor( 11 ) 52769
prime_factor( 2921489 ) 52771
prime_factor( 64753 ) 52769
prime_factor( 14178109 ) 52772
prime_factor( 29 ) 52771
prime_factor( 11 ) 52772
prime_factor( 13 ) 52769
prime_factor( 100741 ) 52771
prime_factor( 4981 ) 52769
prime_factor( 1288919 ) 52772
prime_factor( 17 ) 52769
prime_factor( 293 ) 52769
prime_factor( 43667129 ) 52769
prime_factor( 27853949 ) 52771
prime_factor( 11 ) 52769
prime_factor( 3969739 ) 52769
prime_factor( 29 ) 52771
prime_factor( 557 ) 52769
prime_factor( 960481 ) 52771
prime_factor( 7127 ) 52769
prime_factor( 769 ) 52771
prime_factor( 43960346 ) 52769
prime_factor( 2 ) 52769
prime_factor( 1249 ) 52771
prime_factor( 21980173 ) 52769
prime_factor( 29 ) 52769
prime_factor( 46029309 ) 52771
prime_factor( 3 ) 52771
prime_fa

## Lab: Multiprocessing Pool
* write a program to compute 1!…48! using a multiprocessing pool
* won't be much of a parallelism example, but it's easy to code
* use previous example as a template

1 1
2 2
3 6
4 24
5 120
6 720
7 5040
8 40320
9 362880
10 3628800
11 39916800
12 479001600
13 6227020800
14 87178291200
15 1307674368000
16 20922789888000
17 355687428096000
18 6402373705728000
19 121645100408832000
20 2432902008176640000
21 51090942171709440000
22 1124000727777607680000
23 25852016738884976640000
24 620448401733239439360000
25 15511210043330985984000000
26 403291461126605635584000000
27 10888869450418352160768000000
28 304888344611713860501504000000
29 8841761993739701954543616000000
30 265252859812191058636308480000000
31 8222838654177922817725562880000000
32 263130836933693530167218012160000000
33 8683317618811886495518194401280000000
34 295232799039604140847618609643520000000
35 10333147966386144929666651337523200000000
36 371993326789901217467999448150835200000000
37 13763753091226345046315979581580902400000000
38 523022617466601111760007224100074291200000000
39 20397882081197443358640281739902897356800000000
40 815915283247897734345611269596115894272000000000
41 33

## Multiprocessing Issues/What Else
* primary drawback: sharing data between processes is expensive since all communication between processes requires serialization (pickling) the data
* what we didn't cover
 * futures: objects that wrap threading or multiprocessing depending on what kind of concurrency we need (I/O vs. CPU)
 * AsyncIO: current state of the art in Python concurrent programming